$$\textbf{PLN. Tarea 4: Modelos de Lenguaje Estadísticos}$$
$$\textit{Y. Sarahi García Gozález}$$

<font size=4.5 color='lightblue'>

$\textit{Librerías}$

In [4]:
from nltk.tokenize import TweetTokenizer
import nltk
import numpy as np
import pandas as pd
from math import log10,log2
from collections import Counter
from sklearn import preprocessing
from itertools import combinations
from sklearn.feature_selection import SelectKBest,chi2
import matplotlib.pyplot as plt
import sklearn
from sklearn import svm
from nltk.corpus import stopwords
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.metrics import accuracy_score,confusion_matrix,f1_score,precision_recall_fscore_support,roc_auc_score

In [5]:
print("Tarea realizada en MacOs. \nLas versiones de las librerías y de python utilizadas fueron:\n")
from platform import python_version
print("Python version:", python_version())
print("NumPy version:", np.__version__)
print("NLTK version:", nltk.__version__)
print("Pandas version:", pd.__version__)
print("Scikit-learn version:", sklearn.__version__)

Tarea realizada en MacOs. 
Las versiones de las librerías y de python utilizadas fueron:

Python version: 3.11.0
NumPy version: 1.23.5
NLTK version: 3.8.1
Pandas version: 2.1.4
Scikit-learn version: 1.3.0




<font size=4.5 color='lightblue'>

$\textit{Modelo de lenguaje y evaluación}$

Primero definiremos las funciones para cargar la lista de documentos y los diccionarios necesarios para crear el corpus y el vocabulario.

In [26]:
def get_text_from_file(path_corpus,path_truth):

    tr_txt=[]
    tr_labels=[]

    with open(path_corpus, "r") as f_corpus,open(path_truth, "r") as f_truth:
        for tweet in f_corpus:
            tr_txt += [tweet]
        for label in f_truth:
            tr_labels += [label]   
             
    return tr_txt, tr_labels

def create_dic_freq(corpus,n):
    fdist = nltk.FreqDist(corpus)
    aux=[(fdist[key],key) for key in fdist]
    aux.sort()
    aux.reverse()
    aux=aux[:n]

    return aux

def create_dic_ranking(dic_freq):
    dict_indices=dict()
    cont = 0
    for weight, word in dic_freq:
        dict_indices[word]= cont
        cont+= 1

    return dict_indices





In [ ]:
###### Funcion que crea un corpus con palabras en minusculas y con los caracteres especiales ###### 
###### <s> al inicio del tuit t </s> al final del tuit ###### 
def create_corpus_from_text(tuit_list,tokenizer):

    ''' Funcion que crea un corpus con palabras en minusculas
        y con los caracteres especiales
       <s> al inicio del tuit t </s> al final del tuit
       
        parametros: 
        -lista de tuits (o documentos en general)
        -tokenizador
        returns:
        -corpus con todos los documentos tokenizados (se indica el iniciio y fin de cada uno
         con los caracteres especiales <s>,</s>)
       
    '''

    corpus_palabras = []

    for tuit in tuit_list:
        #minúsculas
        tuit = tuit.lower()
        # token especial <s> al inicio del tuit
        tuit = "<s> " + tuit
        #token especial </s> al final del tuit
        tuit += " </s>"
        #tokenizaos el tuit y agregar las palabras al corpus
        corpus_palabras.extend(tokenizer.tokenize(tuit))

    return corpus_palabras


def create_vocabulary(dictionary_freq,freq_umbral):
    '''
    Función que crea el vocabulario a partir de un diccionario de frecuencias,
    sólo se toman las palabras con frecuencia mayor o igual a freq_umbral
    '''

    #palabras con una frecuencia igual o mayor que el umbral
    vocabulario = [palabra for palabra, frecuencia in dictionary_freq.items() if frecuencia >= freq_umbral]
    #token especial <unk> para las palabras desconocidas
    vocabulario.append("<unk>")
    return vocabulario



def enmascarar(corpus, vocabulario):
    # Enmascarar las palabras desconocidas con <unk>
    corpus_enmascarado = [palabra if palabra in vocabulario else "<unk>" for palabra in corpus]
    
    return corpus_enmascarado

In [7]:
#guardamos los textos de entrenamiento y validación
tr_txt,tr_labels=get_text_from_file("/Users/ely/Documents/Maestria/segundo_semestre/cimat2023-1/lenguaje/practicas/03_practica/mex20_train.txt","/Users/ely/Documents/Maestria/segundo_semestre/cimat2023-1/lenguaje/practicas/03_practica/mex20_train_labels.txt")
val_txt,val_labels=get_text_from_file("/Users/ely/Documents/Maestria/segundo_semestre/cimat2023-1/lenguaje/practicas/03_practica/mex20_val.txt","/Users/ely/Documents/Maestria/segundo_semestre/cimat2023-1/lenguaje/practicas/03_practica/mex20_val_labels.txt")

In [24]:
#Tokenizamos
tokenizer=TweetTokenizer()
#Generamos el corpus
corpus=create_corpus_from_text(tr_txt,tokenizer)

In [27]:
#imprimimos el primer tuit con los tokens especiales d einciio y fin
inicio= corpus.index("<s>")
fin= corpus.index("</s>", inicio)
primer_tuit = " ".join(corpus[inicio : fin+1])

print("Primer tuit del corpus:")
print(primer_tuit)


Primer tuit del corpus:
<s> @usuario @usuario @usuario q se puede esperar del maricon de closet de la yañez aun recuerdo esa ves q lo vi en zona rosa viendo quien lo levantada </s>


<font size=4.5 color='lightblue'>

$\textit{Generación de Texto}$

<font size=4.5 color='lightblue'>

$\textit{El ahorcado}$